In [2]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt 
import seaborn as sns 

# seed 값 설정
seed=0
numpy.random.seed(3)
tf.random.set_seed(3)

Using TensorFlow backend.


In [3]:
# 데이터 불러오기
df = pd.read_csv('data/Automobile_customers.csv')
df.head(10)

,Customer ID,Age,Income_Bucket,Gender,State,Region,Marital_Status,No_of_children,Occupation,Vehicle_Segment,No_of_months,Hobbies,On_Call_Offer
0,46756841,28,2,M,Q,Sub-urban#,1,4,White Collar Clerical,SUV,39,Swimming,Plan 1
1,48408855,25,3,M,A,Sub-urban#,2,3,Engineer,Compact SUV,14,Painting,Plan 5
2,23645631,58,2,M,Z,Urban#,2,4,Defence,Car,24,Reading Books,Plan 5
3,65645152,37,1,M,W,Sub-urban#,2,4,Sales Executive,Pick up truck,27,Gourmet Cooking,Plan 4
4,52236057,64,2,M,S,Sub-urban,1,4,Chef,Pick up truck,56,Music,Plan 4
5,80008322,25,1,M,X,Urban,2,2,Investment Banker,Pick up truck,54,Watching movies,Plan 5
6,12663567,32,1,M,E,Sub-urban,2,1,Orthopeadic,Pick up truck,31,Football,Plan 4
7,62204295,59,3,M,D,Sub-urban,1,3,Gardener,Pick up truck,56,Hockey,Plan 4
8,92845629,56,2,M,C,Urban,1,2,Interior designer,Pick up truck,28,Hiking,Plan 5
9,76395421,38,2,M,R,Sub-urban,1,1,Architect,Pick up truck,46,Para-gliding,Plan 4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10733 entries, 0 to 10732
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Customer ID      10733 non-null  int64 
 1   Age              10733 non-null  int64 
 2   Income_Bucket    10733 non-null  int64 
 3   Gender           10733 non-null  object
 4   State            10715 non-null  object
 5   Region           10733 non-null  object
 6   Marital_Status   10733 non-null  int64 
 7   No_of_children   10733 non-null  int64 
 8   Occupation       10733 non-null  object
 9   Vehicle_Segment  10733 non-null  object
 10  No_of_months     10733 non-null  int64 
 11  Hobbies          10733 non-null  object
 12  On_Call_Offer    10733 non-null  object
dtypes: int64(6), object(7)
memory usage: 1.1+ MB
